# Introduction to ipyautoui

ipyautoui is a Python package that provides a simple way to create interactive user interfaces in Jupyter notebooks. It is built on top of Jupyter widgets and Pydantic and provides a higher-level API for creating interactive widgets.  It is particularly well suited for creating settings panels.


## TO DO IN THIS NOTEBOOK
- [x] Add a brief description of the package and its history?
- [x] Show how to use ipyautoui to create a simple settings panel relatively quickly
  - [ ] Mention the active development of the package and the fact that it is still a bit buggy
  - [ ] Mention other features of Pydantic that may be relevant to our dashboard
- [ ] Have them replace existing settings widgets with control widget generated with ipyautoui
- [ ] Connect new control widget to dashboard

In [ ]:
#| default_exp widgets_pydantic_no_dont

## Making a widget with ipyautoui

1. Define a pydantic model (or write a jason schema)
2. Feed the model to AutoUi
3. You get back a widget! With a value! Whose value is easy to save!

## An example

### Exercise

In the cell below, create a pydantic model called `SimpleModel` with one field, called `window_size`, that is an integer.

In [ ]:
# TODO: write answer 

from pydantic import BaseModel

class SimpleModel(BaseModel):
    window_size : int

To make a widget from this, run the cell below. `AutoUi` takes in the pydantic model and turns it into a widget. Note that it is the class itself, not an instance of the class, that is the argument to `AutoUi`.

In [ ]:
from ipyautoui import AutoUi

ui = AutoUi(SimpleModel)

Run the cell below to display the widget and try changing its value. Note that you can only type numbers (with out decimals) into the box. 

In [ ]:
ui

### Attributes and methods of an AutoUi generated widget

A couple of attributes of the auto-generated widget are particularly useful to know about. 

1. `di_widgets` -- dictionary of widgets, one widget for each pydantic field.
2. `value` -- dictionary of values, one key and value for each pydantic field.

Though one could use `di_widgets` to display the widget for a field outside the auto-generated widget, that is rarely useful. It is sometimes handy, though, to explore the properties of that widget, or to change its properties.

In [ ]:
ui.di_widgets

In this case, `AutoUi` is representing the `int` in our model with an `IntText` widget. That kind of widget only allows integer values to be typed in it.

The `value` for an `AutoUi`-generated widget is a dictionary. The keys are the fields defined in the pydantic model and the values are the current value of that field.

In [ ]:
ui.value

You can set the value, like with any other widget, but make sure that the value is a dictionary.

In [ ]:
ui.value = {"window_size": 11}

The value can also be observed, though there is a twist -- it is `_value` that you observe, not `value`. The observer below simply prints the value of the widget.


In [ ]:
# This observer function just prints to the screen
def print_value(change):
    print(f"{change['new']=}")
    
# NOTE that we are observing changes in _value rather than value
ui.observe(print_value, "_value")

### Exercise

Change the value of `ui` in the cell below and confirm that you get the message you expect.

In [ ]:
# TODO: write answer 

In [ ]:
ui.value = {"window_size": 81}

In [ ]:
ui.value = {"window_size": 84}

Next, let's add Add constraints to window size, one constraint at a time, to see how they affect the widget that is generated.

In [ ]:
from typing import Annotated
from pydantic import Field

class SimpleModel2(BaseModel):
    window_size : Annotated[int, Field(ge=2)]

In [ ]:
ui2 = AutoUi(SimpleModel2)
ui2

Oh my! Apparently the widget is sort of ok with values that are not consistent with the pydantic model. Let's check to see what kind of widget ipyautoui made for us:

In [ ]:
ui2.di_widgets

It seems like a `BoundedIntText` might be a better representation of this field. We can tell ipyautoui to use that widget via a dictionary passed into the `json_schema_extra` argument. The dictionary key that specifies the widget is `autoui`.

In [ ]:
class SimpleModel3(BaseModel):
    window_size : Annotated[int, Field(ge=2, json_schema_extra=dict(autoui='ipywidgets.BoundedIntText'))]

In [ ]:
ui3 = AutoUi(SimpleModel3)
ui3

Nice, now the user cannot set the value below 2! As you can see below, that is because the `min` attribute of the `BoundedIntText` is set to 2.

In [ ]:
ui3.di_widgets

Next we add the constraint that the window size must be less than or equal to 100. We also use the `desription` argument to `Field` to add a brief description of the field to the user interface.

In [ ]:
class SimpleModel4(BaseModel):
    window_size: Annotated[int, Field(ge=2, le=100, description="Size of smoothing window")]

In [ ]:
ui4 = AutoUi(SimpleModel4)
ui4

Note that we have also let `ipyautoui` choose the widget -- with both an upper and lower limit provided it makes a slider.

In [ ]:
SimpleModel4.model_json_schema()

In [ ]:
SimpleModel2.model_json_schema()

## The data selector

Let's import the pydantic model we finished up the last notebook with.

In [ ]:
from ipyautoui import AutoUi
from dashboard.widgets_pydantic import DataSelectorModelDraft6

Next, we create a UI for the model.

In [ ]:
controls = AutoUi(DataSelectorModelDraft6)
controls

That looks almost right, except that we want the year range selector to be a slider, not a....whatever that is.

In [ ]:
from pydantic import BaseModel, Field, model_validator
from typing import Annotated

### Exercise 

Add the necessary code to the class definition below to force the `year_range` to be represented with an `IntRangeSlider`.

In [ ]:
# TODO: write answer 

class DataSelectorModelPenultimate(BaseModel, validate_assignment=True):
    year_range: Annotated[
        tuple[int, int],
        Field(json_schema_extra=dict(autoui='ipywidgets.IntRangeSlider'))  # Fill in the correct arguments here
    ] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

In [ ]:
ui = AutoUi(DataSelectorModelPenultimate)
ui

### Exercise

Try setting the window size and polynomial order to an invalid combination and see what happens.

### Fixing the range of the slider

You may have noticed that range of the `year_range` slider isn't correct. The minimum is 0 and the maximum is 100.

There are a couple of ways we can fix this:
 
1. Set those properties on the slider.
2. Modify the type hint we provide for `year_range`.

Let's try both. For the sake of argument we will assume that the year range is 1880 to 2020.

#### Setting the widget properties

Recall that the `di_widgets` attribute of an `AutoUi` control is a dictionary whose keys are our field names and whose values are the widget(s) representing that field.

With that in mind, we can set the `min` and `max` properties of the auto-generated control like this:

In [ ]:
ui.di_widgets["year_range"].max = 2020
ui.di_widgets["year_range"].min = 1880

We can set the value in a similar way.

In [ ]:
ui.di_widgets["year_range"].value = (1880, 2020)

#### Modifying the type hints

An alternative approach to setting the `min` and `max` directly is to modify the pydantic model to reflect the fact that the two `int`s that make up the `year_range` have constraints on them: each of the integers must fall within the range of years provided.

What we want to end up doing is expressing to pydantic that each `int` has a restricted range. We'll do that by defining our own type below and then using that custom type in our Ppydantic model.

We use feature of type hinting introduced in Python 3.12 called type aliasing to do this. The feature looks very different in Python 3.12 than it does in ealier versions of Python.

The cell below imports`TypeAliasType` from `typing-extensions`to let pydantic and type checkers know that we are defining a new type. `typing-extensions` is a package that backports new typing features in python to Python versions before the feature was added.

The equivalent to the definition of `ConstrainedInt` below in Python 3.12 looks like this:

```python
# Only works in Python 3.12 and up!
type ConstrainedInt = Annotated[int, Field(ge=1880, le=2020)]
```

In [ ]:
from typing_extensions import TypeAliasType

In [ ]:
ConstrainedInt = TypeAliasType("ConstrainedInt", Annotated[int, Field(ge=1880, le=2020)])

In [ ]:
class DataSelectorModel(BaseModel, validate_assignment=True):
    year_range: Annotated[
        # The key change is in the line below
        tuple[ConstrainedInt, ConstrainedInt],
        # With this change to the type we no longer need to tell ipyautoui
        # what kind of widget to use. Field contains just a brief description
        Field(description="Range of years to plot")
    ] = (1800, 2000)
    window_size: Annotated[int, Field(ge=2, le=100)] = 2
    polynomial_order: Annotated[int, Field(ge=1, le=10)] = 1

    # mode="after" means the validator runs after pydantic has checked that the individual
    # fields have values that are valid.
    @model_validator(mode="after")
    def limit_polynomial_order(self):
        
        if self.polynomial_order > self.window_size - 1:
            # Handle a bad polynomial order or window size
            raise ValueError("Polynomial order must be smaller than window size")
            
        # If we got this far the polynomial order is consistent with the window size
        # so return self. Failing to return self will end up causing an error.
        return self

In [ ]:
ui_final = AutoUi(DataSelectorModel)
ui_final

## Enforcing the constraint between polynomial order and window size

There are a few options here:

1. Change one or both of the ranges of `widnow_size` and `polynomial_order`
    1. When window size is being changed and a conflict arises, either
        1. update the maximum value of the polynomial order to be consistent with window size, with the side effect that the value will be updated to.
        1. update the value of `polynomial_order` but not its maximum.
    1. When the polynomial order is changed, either
        1. update the minimum allowed window size, or
        1. update the value of the window size.
1. reset the controls to a valid value and display a warning to the user.

It does not matter so much which of these you choose, but you do need to choose one.

In the first version of the dashboard we did choice 1.A.a -- when `window_size` changed the `polynomial_order` was set to the smaller of 10 and `window_size - 1`.

Here we will implement 2 because it is a little different than what we did in the first round of the dashboard. It also places the responsibility of deciding the correct course of action (increase `window_size` or decrease `polynomial-order`) on the user. 

Our approach is to observe the `_value` of the widget, try to make a valid model out of it, and if that fails set the widget to the old value. `ipyautoui` will take care of displaying an appropriate error message for us.

The observe below is created using a [*closure*](https://en.wikipedia.org/wiki/Closure_(computer_programming)), which is a function that is created inside of another function where the inner function uses some of the variables form the outer function. Here we do that instead of treating `ui` as a global variable.

In [ ]:
from pydantic import ValidationError
def make_enforcer(ui):
    """
    Make a function that can be used to observe changes on a 
    user interface element.

    Parameters
    ----------

    ui: an AutoUi widget

    Returns
    -------

    callable
        A function that can be used as the observer of a traitlets event.
    """
    def constraint_enforcer(change):
        """
        Reset widget to the most recent valid value if the new
        value results in an invalid value.
        """
        try:
            # Every AutoUi widget has a copy of the model class
            # We'll try validating the value in change["new"] and see if it works
            ui.model.model_validate(change["new"])
        except ValidationError:
            # That failed, so reset the ui to the old value
            ui.value = change["old"]

    return constraint_enforcer

In [ ]:
ui_final.observe(make_enforcer(ui_final), "_value")

In [ ]:
ui_final

## Further exploration of ipyautoui

The easiest way to get a better idea of what ipyautoui can do is to try out its demo. If you have time, feel free to explore!

In [ ]:
from ipyautoui import demo
demo()